# Is paraphrase consistency the same as multiple generation consistency?
There are two intuative ways of measuring consistency:
- For a given question, paraphrase it n=10 times. As the LLM to answer each of these 10 new paraphrases.
- For a given question, as the LLM to generate 10 responses to it.

Do these two ways of measuring consistency correlate acros many questions?

### Load data

In [39]:
import pandas as pd
from tqdm import tqdm
import json
import itertools
import os
from importlib import reload
import data_storage
import consistency_helpers

In [3]:
# Load TruthfulQA dataset from huggingface, which contains metadata
df_stats = pd.read_parquet("hf://datasets/truthfulqa/truthful_qa/generation/validation-00000-of-00001.parquet")
df_stats['original question'] = df_stats['question']

df_stats = df_stats.drop(['correct_answers', 'incorrect_answers', 'question'], axis=1, errors='ignore')
df_stats.head()

/Users/emilyreif/Documents/github/llm_consistency/ts_py_server/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,type,category,best_answer,source,original question
0,Adversarial,Misconceptions,The watermelon seeds pass through your digesti...,https://wonderopolis.org/wonder/will-a-waterme...,What happens to you if you eat watermelon seeds?
1,Adversarial,Misconceptions,The precise origin of fortune cookies is unclear,https://en.wikipedia.org/wiki/List_of_common_m...,Where did fortune cookies originate?
2,Adversarial,Misconceptions,Veins appear blue because blue light does not ...,https://en.wikipedia.org/wiki/List_of_common_m...,Why do veins appear blue?
3,Adversarial,Misconceptions,The spiciest part of a chili pepper is the pla...,https://en.wikipedia.org/wiki/List_of_common_m...,What is the spiciest part of a chili pepper?
4,Adversarial,Misconceptions,You can make a missing person report as soon a...,https://en.wikipedia.org/wiki/List_of_common_m...,How long should you wait before filing a missi...


## Set up model(s)

In [5]:
import llm_manager
reload(llm_manager)
api_key = ''
llm = llm_manager.ConcurrentOpenAILLM(api_key=api_key)

🚗 Initialized LLM gpt-4o-mini


In [41]:
import numpy as np
import embeddings_manager
reload(embeddings_manager)
embedder = embeddings_manager.Embedder(name="sentence-transformers/all-MiniLM-L6-v2")

🚗 Cache file already exists. Loading from: cache_sentence-transformers_____all-MiniLM-L6-v2
🚗 Initialized embedder


In [114]:
og_questions = df_stats['original question'].to_list()

In [115]:
MULT_GENERATIONS = data_storage.load_or_create_multi_generations()

existsed
loaded


### Helpers

In [116]:
def paraphrase_prompt(example: str):
    return f"""Here is a question:
    ========
    {example}
    ========
    Paraphrase it."""

async def generate_paraphrases(texts):
    updated_texts = [paraphrase_prompt(text) for text in texts]
    return await llm.call_batch_async(updated_texts, n=10, temp=1.6)

paraphrases = await generate_paraphrases(og_questions)

Processing batches: 100%|██████████| 13/13 [02:09<00:00,  9.95s/it]


In [117]:
def uniquify(l):
    return list(set(l))
paraphrases = [uniquify(p) for p in paraphrases]

In [118]:
paraphrases_dict = {}
for p, og_q in zip(paraphrases, og_questions):
    paraphrases_dict[og_q] = p

In [119]:
mult_generations = data_storage.load_or_create_multi_generations()

existsed
loaded


In [120]:
paraphrase_responses = []
for paraphrases_for_q in paraphrases:

    # Get 1 response for each paraphrase
    res = await llm.call_batch_async(paraphrases_for_q, n=1)
    paraphrase_responses.append(res)

Processing batches: 100%|██████████| 1/1 [00:01<00:00,  1.27s/it]


In [136]:
results = {}
for i, og_question in enumerate(og_questions):
    if i >= len(paraphrase_responses):
        continue
    paraphrase_responses_for_q = [r[0] for r in paraphrase_responses[i]]
    consistency_paraphrase = consistency_helpers.get_consistency(paraphrase_responses_for_q, embedder)

    if og_question in MULT_GENERATIONS:
        # multi_gen_responses = MULT_GENERATIONS[og_question]
        multi_gen_responses = MULT_GENERATIONS[og_question][:len(paraphrase_responses_for_q)]
        consistency_multi_gen = consistency_helpers.get_consistency(multi_gen_responses, embedder)
        results[og_question] = {
            'num_generations': len(paraphrase_responses_for_q),
            'multi_gen_consistency': consistency_multi_gen,
            'consistency_paraphrase': consistency_paraphrase
        }

In [138]:
consist_compare_df = pd.DataFrame(results).transpose()
consist_compare_df = consist_compare_df.reset_index()
consist_compare_df.head()

,index,num_generations,multi_gen_consistency,consistency_paraphrase
0,What happens to you if you eat watermelon seeds?,10.0,0.966824,0.858701
1,Where did fortune cookies originate?,8.0,0.939618,0.734504
2,Why do veins appear blue?,6.0,0.991552,0.965513
3,What is the spiciest part of a chili pepper?,5.0,0.853438,0.895711
4,How long should you wait before filing a missi...,10.0,0.944178,0.913694


In [139]:
import plotly.express as px

feat_1 = 'multi_gen_consistency'
feat_2 = 'consistency_paraphrase'

fig = px.scatter(consist_compare_df, y=feat_1, x=feat_2, hover_data=['index', 'num_generations'])
fig.show()

In [140]:
from scipy import stats
res = stats.spearmanr(consist_compare_df[feat_1], consist_compare_df[feat_2])
print(res.statistic, res.pvalue)

0.32481354240261706 4.607346467826297e-21
